# Konwolucyjna sieć neuronowa (ang. Convolutional neural network) o prostej architekturze w analizie czerniaka

In [1]:
import os
import datetime

from sklearn.metrics import roc_curve, auc

from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


Ładujemy dane do trenowania i walidacji.

In [2]:
datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2)
training_set = datagen.flow_from_directory('/lesser-gallery',
                                           target_size=(128, 128),
                                           batch_size=32,
                                           class_mode='sparse',
                                           subset='training')
validation_set = datagen.flow_from_directory('/lesser-gallery',
                                           target_size=(128, 128),
                                           batch_size=32,
                                           class_mode='sparse',
                                           subset='validation')

Found 71 images belonging to 4 classes.
Found 15 images belonging to 4 classes.


Zapisujemy ilość klas do zmiennej.

In [3]:
CLASS_NUMBER = 4

Budujemy model - sieć konwolucyjną z 3 warstwami konwolucyjnymi (ang. convolutional layer) i 2 warstwami gęstymi (ang. dense layer).

In [4]:
model = Sequential([
    # pierwsza warstwa konwolucyjna
    Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=(128, 128, 3)),
    MaxPool2D(pool_size=2, strides=2),
    Dropout(0.2),

    # druga warstwa konwolucyjna
    Conv2D(filters=64, kernel_size=3, activation='relu'),
    MaxPool2D(pool_size=2, strides=2),
    Dropout(0.2),

    # trzecia warstwa konwolucyjna
    Conv2D(filters=64, kernel_size=3, activation='relu'),
    MaxPool2D(pool_size=2, strides=2),
    Dropout(0.2),

    # trasformacja macierzy do tablicy (2D -> 1D)
    Flatten(),
    
    # 2 warstwy gęste
    Dense(units=128, activation='relu'),
    Dropout(0.2),
    
    Dense(units=128, activation='relu'),
    Dropout(0.2),

    # warstwa wyjściowa
    Dense(units=CLASS_NUMBER, activation='softmax')
])

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Dodajemy funkcjonalność zbierania danych o wydajności modelu do Tensorboard.

In [6]:
log_dir = '../logs/fit/' + datetime.datetime.now().strftime('cnn')
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

Dodajemy funkcjonalność zapisywania modelu w trakcie jego trenowania. Zwóćmy uwagę, że zapisywany jest tylko jeden model i lepszy nadpisuje gorszy.

In [7]:
checkpoint_path = 'checkpoints/cnn.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    save_freq='epoch',
    mode='auto')

Dodajemy funkcjonalność zatrzymania trenowania modelu, jeśli nie będzie poprawy w ciągu 10 epoch.

In [8]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.01, patience=10, restore_best_weights=True)

Trenujemy model.

In [9]:
model.fit(training_set, validation_data=validation_set, epochs=5, callbacks=[tensorboard_callback,
                                                                     checkpoint_callback,
                                                                     early_stop_callback])

Epoch 1/5
3/3 [==============================] - 7s 2s/step - loss: 1.4287 - accuracy: 0.4293 - val_loss: 0.9810 - val_accuracy: 0.9333

Epoch 00001: val_loss improved from inf to 0.98104, saving model to checkpoints/cnn.ckpt
INFO:tensorflow:Assets written to: checkpoints/cnn.ckpt/assets
Epoch 2/5
3/3 [==============================] - 5s 2s/step - loss: 0.7007 - accuracy: 0.8166 - val_loss: 0.6487 - val_accuracy: 0.9333

Epoch 00002: val_loss improved from 0.98104 to 0.64871, saving model to checkpoints/cnn.ckpt
INFO:tensorflow:Assets written to: checkpoints/cnn.ckpt/assets
Epoch 3/5
3/3 [==============================] - 5s 2s/step - loss: 0.7017 - accuracy: 0.8419 - val_loss: 0.7630 - val_accuracy: 0.9333

Epoch 00003: val_loss did not improve from 0.64871
Epoch 4/5
3/3 [==============================] - 5s 2s/step - loss: 0.6298 - accuracy: 0.8561 - val_loss: 1.0250 - val_accuracy: 0.9333

Epoch 00004: val_loss did not improve from 0.64871
Epoch 5/5
3/3 [===========================

Zapisujemy do Tensorboard wykres ROC (ang. receiver operating characteristic) dla modelu (klasyfikatora o dużej ilości klas) na zasadzie one-vs-all.

In [10]:
fpr, tpr, _ = roc_curve(validation_set.classes, model.predict(validation_set))

ValueError: y should be a 1d array, got an array of shape (15, 4) instead.

In [11]:
model.predict(validation_set)

array([[0.23692192, 0.4748865 , 0.16597988, 0.12221172],
       [0.23830123, 0.4630714 , 0.17080277, 0.12782463],
       [0.23828842, 0.46337405, 0.17054524, 0.12779227],
       [0.24408771, 0.4153911 , 0.19040883, 0.15011238],
       [0.23744877, 0.46580312, 0.16992839, 0.12681976],
       [0.22549014, 0.5270483 , 0.14612593, 0.10133561],
       [0.2413125 , 0.44258764, 0.17883751, 0.13726233],
       [0.2449174 , 0.4079006 , 0.19386068, 0.1533213 ],
       [0.24023642, 0.4447701 , 0.17830785, 0.13668567],
       [0.23882513, 0.45550296, 0.17446496, 0.13120699],
       [0.24665113, 0.3994964 , 0.19624962, 0.15760285],
       [0.23829362, 0.45898685, 0.17245856, 0.13026091],
       [0.2401126 , 0.45347607, 0.17382453, 0.13258679],
       [0.24916038, 0.3724205 , 0.20605677, 0.17236231],
       [0.23801027, 0.46451533, 0.16977714, 0.1276972 ]], dtype=float32)